In [3]:
import pandas as pd
import os
import csv
import numpy as np
import matplotlib.pyplot as plt

income_file = "irs_data/Income Data.csv"
demo_file = "irs_data/Demographics.csv"

income_df = pd.read_csv(income_file)
demo_df = pd.read_csv(demo_file)




FileNotFoundError: [Errno 2] File irs_data/Income Data.csv does not exist: 'irs_data/Income Data.csv'

In [2]:
income_df.head()
demo_df.head()

NameError: name 'income_df' is not defined

In [ ]:
zip_df = income_df.loc[1:, ["NAME", "DP03_0062E"]]
zip_df["PostalCode"] = zip_df["NAME"].str.split(" ").str[-1]
zip_df.columns =["drop", "Median_Income", "PostalCode"]
zip_income = zip_df[["PostalCode", "Median_Income"]]
zip_income.head()


In [ ]:
demo_zip = demo_df.loc[1:, ["NAME", "DP05_0001E", "DP05_0009E", "DP05_0010E", "DP05_0018E"]]
demo_zip["PostalCode"] = demo_df["NAME"].str.split(" ").str[-1]
demo_zip.columns = ["NAME", "Population", "20to24", "25to34", "Median_age", "PostalCode" ]
demo_zip = demo_zip.loc[(demo_zip["Population"].astype(str).astype(int)>0)]
demo_zip["Keydemo_per"] = (demo_zip["20to24"].astype(str).astype(int) + demo_zip["25to34"].astype(str).astype(int)) / demo_zip["Population"].astype(str).astype(int)
demo_zip = demo_zip[["PostalCode", "Population", "Median_age", "Keydemo_per"]]


In [ ]:
zipcode_data = pd.merge(zip_income, demo_zip, on = "PostalCode")
zipcode_data.head()

In [ ]:
zipcode_data.to_csv("income_data.csv", index = False)

In [ ]:
income_file = "income_clean.csv"
income_df = pd.read_csv(income_file)
income_df.dtypes

In [ ]:
income_df["Median_age"]= pd.to_numeric(income_df['Median_age'],errors='coerce')
income_df["Median_Income"]= pd.to_numeric(income_df['Median_Income'],errors='coerce')

In [ ]:
income_df.dtypes

In [ ]:
income_df.to_csv("income_data.csv", index = False)

In [ ]:
income_df= income_df.loc[:, ["PostalCode", "Median_Income", "Population", "Median_age", "Keydemo_per"]]
income_df.head()

In [ ]:
income_df.to_csv("income_data.csv", index = False)

In [ ]:
income_df.head()

In [ ]:
income_file = "income_data.csv"
income_df = pd.read_csv(income_file)
brewery_file = "../breweries_data/brewery_clean.csv"
brewery_df = pd.read_csv(brewery_file)

In [ ]:
brewery_df = brewery_df.drop(columns=['Unnamed: 0'])
brewery_df.head()

In [ ]:
brewery_count = brewery_df.groupby('PostalCode').agg({'Name': ['count']})
brewery_count.head()

In [ ]:
brewery_count_data = pd.merge(brewery_df, brewery_count, how = 'right', on = "PostalCode")

In [ ]:
brewery_income = pd.merge(income_df, brewery_count, how = 'right', on = "PostalCode")
brewery_income.columns = ["PostalCode", "Median_Income", "Population", "Median_age", "Keydemo_per", "Brewery_count"]
brewery_income.head()

In [ ]:
sorted_brewery = brewery_income.sort_values(by='Brewery_count', ascending = False)
sorted_brewery.reset_index(drop=True, inplace=True)
sorted_brewery.head()

In [ ]:
starbucks_file = "../starbucks/starbucks_clean.csv"
starbucks_df = pd.read_csv(starbucks_file)
starbucks_df = starbucks_df.drop(columns=['Unnamed: 0'])
starbucks_df["PostalCode"]= starbucks_df["PostalCode"].fillna(0.0).astype(int)
sb_count = starbucks_df.groupby('PostalCode').agg({'PostalCode': ['count']})
sb_count.columns = ['Starbucks_count']
sb_sorted = sb_count.sort_values(by = 'Starbucks_count', ascending = False)
sb_sorted.head()

In [ ]:
combined_df = pd.merge(sorted_brewery, sb_sorted, how = 'left', on = "PostalCode")
combined_df["pop_per_brewery"] = combined_df["Population"]/ combined_df["Brewery_count"].apply(lambda x: x if x > 0 else 1)
combined_df["pop_per_starbucks"] = combined_df["Population"]/ combined_df["Starbucks_count"].apply(lambda x: x if x > 0 else 1)
combined_df['Brewery_rank'] = combined_df['pop_per_brewery'].rank(na_option='bottom', ascending = False)
combined_df['Income_rank'] = combined_df['Median_Income'].rank(na_option='bottom', ascending = False)
combined_df['Demo_rank'] = combined_df['Keydemo_per'].rank(na_option='bottom', ascending = False)
combined_df['Starbucks_rank'] = combined_df['pop_per_starbucks'].rank(na_option='bottom')
combined_df['combined_score'] = combined_df['Brewery_rank'] + combined_df['Income_rank'] + combined_df['Demo_rank'] + combined_df['Starbucks_rank']
combined_df['Total_rank'] = combined_df['combined_score'].rank(na_option='bottom')
ranked_df = combined_df.sort_values(by = 'Total_rank')

In [ ]:
Top100_zip = ranked_df.head(100)
Top100_zip

In [ ]:
Top100_zip.to_csv("Top100_zip.csv", index = False)

In [ ]:
Top100_zip.plot(kind="scatter", x="Keydemo_per", y="pop_per_brewery", c ="Median_Income", colormap='viridis', grid=True, figsize=(18,16),
              title="People per Brewery vs Key Demo % )
plt.show()
plt.savefig('Top100.png', bbox_inches='tight')